# Моделирование оценочных систем
Вариант 3

In [150]:
import numpy as np
from random import randint
from typing import Callable
from scipy.linalg import eigh

## Задача 1

Дан следующий массив с названиями объектов, и пусть выделено 5 главных признаков:

In [121]:
given = np.array([
    [2,3,2,2,2,1],
    [3,4,4,3,3,3],
    [2,3,1,3,3,1],
    [3,2,2,3,3,2],
    [2,4,1,2,2,2],
    [3,3,4,2,2,3],
    [2,3,3,3,2,2]
])

labels = {
    0 : "Новые окна",
    1 : "ЕвроОКНА",
    2 : "Русские окна",
    3 : "Горизонт",
    4 : "ViPlast",
    5 : "Velux",
    6 : "Витраж"
}

idx = [0,1,2,3,4]

print("We have next labels:\n{}\nAnd this table:\n{}\nMain features \
are ones that are in next columns: {}".format(labels,given,idx))

We have next labels:
{0: 'Новые окна', 1: 'ЕвроОКНА', 2: 'Русские окна', 3: 'Горизонт', 4: 'ViPlast', 5: 'Velux', 6: 'Витраж'}
And this table:
[[2 3 2 2 2 1]
 [3 4 4 3 3 3]
 [2 3 1 3 3 1]
 [3 2 2 3 3 2]
 [2 4 1 2 2 2]
 [3 3 4 2 2 3]
 [2 3 3 3 2 2]]
Main features are ones that are in next columns: [0, 1, 2, 3, 4]


Так как показатели устроены по принципу "чем больше, тем лучше", то нормировать будем по следующему принципу:
$$ x_i^{new} = \frac{x_i - x_i^{min}}{x_i^{max} - x_i^{min}} $$

In [122]:
def normalize(arr: np.array, type = 'min') -> np.array:
    """
    Perform normalization of given matrix
    """
    res = []
    for line in np.transpose(arr):
        xmin = line.min()
        xmax = line.max()
        if type == 'min':
            f = lambda x: (x - xmin) / ( xmax - xmin )
        else:
            f = lambda x: (xmax - x) / ( xmax - xmin )
        res.append([round(f(x),2) for x in line])
    return np.transpose(np.array(res))

In [126]:
normed = normalize(given)
work = np.take(normed,idx,axis=1)
np.savetxt("data/normed.csv", normed, delimiter=' & ', fmt='%1.1f', newline=' \\\\\n')
print("Normalized table is:\n{}".format(normed))

Normalized table is:
[[0.   0.5  0.33 0.   0.   0.  ]
 [1.   1.   1.   1.   1.   1.  ]
 [0.   0.5  0.   1.   1.   0.  ]
 [1.   0.   0.33 1.   1.   0.5 ]
 [0.   1.   0.   0.   0.   0.5 ]
 [1.   0.5  1.   0.   0.   1.  ]
 [0.   0.5  0.67 1.   0.   0.5 ]]


### Метод парных сравнений
Применим метод парных сравнений, сначала создав матрицу парных сравнений:

In [164]:
def pc_matrix(arr:np.array, idx=None) -> np.array:
    if idx is None:
        idx = np.linspace(0,4,5).astype('int64')
    pc_arr = np.take(arr,idx, axis=1)
    pc_matrix = np.zeros((pc_arr.shape[1], pc_arr.shape[1]))
    for i in range(pc_matrix.shape[0]):
        for j in range(pc_matrix.shape[1]):
            if i <= j:
                pc_matrix[i,j] = randint(0,2)
            else:
                pc_matrix[i,j] = 2 - pc_matrix[j,i]
                
    return pc_matrix

def paired_comparison(arr:np.array, eps = 10e-5, iterly = True) -> np.array:
    """
    Perform paired comparison method
    """
    if iterly:
        v = arr.sum(axis=1).reshape(-1,1)
        k = 0
        found = False
        prev_w = 0
        while not found:
            w = v / v.sum()
            if k != 0:
                if (np.abs(w - prev_w) < eps).all():
                    found = True
                    w_star = w
            k += 1
            prev_w = w
            v = arr.dot(v)
    else:
        _,v = eigh(arr)
        w_star = v[:,-1]
        
    return w_star

In [153]:
pc_mat = pc_matrix(given, idx=idx)
np.savetxt("data/pc_mat.csv", pc_mat, delimiter=' & ', fmt='%1.1f', newline=' \\\\\n')
print("We got next paired comparison matrix:\n{}".format(pc_mat))
weights = paired_comparison(pc_mat)
print("We got next weights:\n{}".format(weights))

We got next paired comparison matrix:
[[2. 1. 2. 2. 0.]
 [1. 0. 2. 2. 1.]
 [0. 0. 2. 0. 2.]
 [0. 0. 2. 1. 2.]
 [2. 1. 0. 0. 2.]]
We got next weights:
[[0.25655311]
 [0.21013577]
 [0.13844433]
 [0.17115931]
 [0.22370748]]


In [165]:
w = paired_comparison(pc_mat, iterly=False)
w

array([0.67388734, 0.30290545, 0.        , 0.        , 0.67388734])

Будем использовать в качестве операции агрегирования взвешенное среднее:
$$ S(x,w) = <x,w> $$

In [166]:
def weighted_avg(x:np.array, w:np.array) -> np.double:
    """
    Computes weighted average
    """
    return np.dot(x,w)

In [170]:
aggrd = np.apply_along_axis(weighted_avg,1,work, weights)
print("So ranking is:\n{}".format(aggrd))

So ranking is:
[[0.15075451]
 [1.        ]
 [0.49993468]
 [0.69710653]
 [0.21013577]
 [0.50006532]
 [0.3689849 ]]


In [171]:
for m in aggrd:
    print("{:.2f}".format(*m))

0.15
1.00
0.50
0.70
0.21
0.50
0.37


А значит лучший телефонный оператор:

In [168]:
labels[aggrd.argmax()]

'ЕвроОКНА'

### OWA-оператор
Теперь решим данную задачу с помощью OWA:

In [174]:
def OWA_weights(arr:np.array, func : Callable) -> np.array:
    n = arr.shape[1]
    w = np.zeros(n)
    w[0] = func(1/(n))
    for i in range(1,n):
        w[i] = func((i+1)/n) - func(i/n)
        
    return w

def OWA(arr: np.array, alpha = 2) -> np.array:
    q = lambda x: x**alpha
    weights = OWA_weights(arr,q)
    print(weights)
    srtd_arr = np.apply_along_axis(sorted,1,arr)
    return np.apply_along_axis(np.dot,1,srtd_arr,weights)

In [175]:
owa_aggrd = OWA(work)
print("So ranking is:\n{}".format(owa_aggrd))

[0.04 0.12 0.2  0.28 0.36]
So ranking is:
[0.2724 1.     0.74   0.8796 0.36   0.74   0.6476]


Значит лучший оператор:

In [176]:
labels[owa_aggrd.argmax()]

'ЕвроОКНА'

## Задача 2

In [177]:
given = np.array([
    ['VL', 'H', 'L', 'M', 'M', 'VH'],
    ['L', 'M', 'L', 'M', 'M', 'M'],
    ['H', 'L', 'M', 'H', 'H', 'H'],
    ['M', 'VH', 'H', 'H', 'H', 'M'],
    ['H', 'M', 'VH', 'H', 'M', 'VH'],
    ['M', 'M', 'H', 'M', 'L', 'M'],
    ['M', 'L', 'M', 'H', 'M', 'H'],
    ['VH', 'H', 'M', 'M', 'L', 'M']
])

labels = {
    0 : "Аврора",
    1 : "Кристалл",
    2 : "Энергия",
    3 : "Здоровый город",
    4 : "Леди N",
    5 : "Университетский",
    6 : "Академия красоты и здоровья",
    7 : "Филиал №6"
}

idx = [0,1,2,3]
work = np.take(given,idx,axis=1)

terms_interpret = {
    'VL' : 0,
    'L' : 1,
    'M' : 2,
    'H' : 3,
    'VH' : 4
}

print("We have next labels:\n{}\nAnd this table:\n{}\nMain features \
are ones that are in next columns: {}".format(labels,given,idx))

We have next labels:
{0: 'Аврора', 1: 'Кристалл', 2: 'Энергия', 3: 'Здоровый город', 4: 'Леди N', 5: 'Университетский', 6: 'Академия красоты и здоровья', 7: 'Филиал №6'}
And this table:
[['VL' 'H' 'L' 'M' 'M' 'VH']
 ['L' 'M' 'L' 'M' 'M' 'M']
 ['H' 'L' 'M' 'H' 'H' 'H']
 ['M' 'VH' 'H' 'H' 'H' 'M']
 ['H' 'M' 'VH' 'H' 'M' 'VH']
 ['M' 'M' 'H' 'M' 'L' 'M']
 ['M' 'L' 'M' 'H' 'M' 'H']
 ['VH' 'H' 'M' 'M' 'L' 'M']]
Main features are ones that are in next columns: [0, 1, 2, 3]


Сформируем вектор весов так же, как и формировали вектор весов для OWA-оператора, а также отсортируем оценки объектов по выбранным показателям по невозрастанию с учетом интерпретации термов:

In [178]:
def lowa_sort(arr:np.array, scale: dict) -> np.array:
    """
    Performs sort using linguistic scale
    """
    lst = list(arr)
    for i in range(len(l)):
        lst[i] = sorted(lst[i], key=lambda x: scale[x])
    return np.array(lst)

In [182]:
weights = OWA_weights(work, func = lambda x: x**2)
srtd_vals = lowa_sort(work,terms_interpret)
print("Weights are:\n{}\nSorted marks are:\n{}".format(weights,srtd_vals))

Weights are:
[0.0625 0.1875 0.3125 0.4375]
Sorted marks are:
[['VL' 'L' 'M' 'H']
 ['L' 'L' 'M' 'M']
 ['L' 'M' 'H' 'H']
 ['M' 'H' 'H' 'VH']
 ['M' 'H' 'H' 'VH']
 ['M' 'M' 'M' 'H']
 ['L' 'M' 'M' 'H']
 ['M' 'M' 'H' 'VH']]


Применим $LOWA$-оператор и получим ранжирование объектов по данным оценкам:

In [183]:
def lowa(arr:np.array, weights:np.array, scale:dict) -> np.array:
    """
    Applies LOWA-operator to given data
    """
    n = weights.shape[0]
    rnkg_arr = [0]*arr.shape[0]
    T = len(scale) - 1
    if n == 2:
        for it in range(arr.shape[0]):
            j = scale[arr[it,0]]
            i = scale[arr[it,1]]
            k = min(T,i + round(weights[0]*(j-1)))
            rnkg_arr[it] = list(scale.keys())[list(scale.values()).index(k)]
    else:
        l = weights[1:] / weights[1:].sum()
        right_term = lowa(arr[:,1:], l,scale)
        for it in range(arr.shape[0]):
            j = scale[arr[it,0]]
            i = scale[right_term[it]]
            k = min(T,i + round(weights[0]*(j-1)))
            rnkg_arr[it] = list(scale.keys())[list(scale.values()).index(k)]
            
    return np.array(rnkg_arr)

In [184]:
vals = lowa(srtd_vals, weights, terms_interpret)
for i in range(len(labels)):
    print(" {} : {}".format(labels[i], vals[i]))

 Аврора : H
 Кристалл : M
 Энергия : VH
 Здоровый город : VH
 Леди N : VH
 Университетский : H
 Академия красоты и здоровья : H
 Филиал №6 : VH


Сделаем ранжирование объектов:

In [185]:
pairs = dict(zip(labels,vals))
ranking = {labels[k] : v for k,v in sorted(pairs.items(), 
                                           key = lambda item : terms_interpret[item[1]], 
                                           reverse = True)}
for k,v in ranking.items():
    print(k,v)

Энергия VH
Здоровый город VH
Леди N VH
Филиал №6 VH
Аврора H
Университетский H
Академия красоты и здоровья H
Кристалл M
